# अध्याय ७: चॅट अनुप्रयोग तयार करणे
## OpenAI API जलद प्रारंभ

हा नोटबुक [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) मधून घेतला आहे, ज्यामध्ये [Azure OpenAI](notebook-azure-openai.ipynb) सेवांमध्ये प्रवेश करणारे नोटबुक्स समाविष्ट आहेत.

Python OpenAI API Azure OpenAI मॉडेल्ससह देखील कार्य करते, काही बदलांसह. येथे फरकांबद्दल अधिक जाणून घ्या: [Python वापरून OpenAI आणि Azure OpenAI एंडपॉइंट्समध्ये कसे स्विच करावे](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)


# परिचय  
"मोठ्या भाषा मॉडेल्स म्हणजे मजकूर ते मजकूर नकाशा करणारे फंक्शन्स आहेत. दिलेल्या इनपुट मजकूराच्या स्ट्रिंगवर, मोठे भाषा मॉडेल पुढे कोणता मजकूर येईल हे भाकीत करण्याचा प्रयत्न करतात"(1). हा "क्विकस्टार्ट" नोटबुक वापरकर्त्यांना उच्च-स्तरीय LLM संकल्पना, AML सुरू करण्यासाठी आवश्यक मुख्य पॅकेजेस, प्रॉम्प्ट डिझाइनची साधी ओळख, आणि विविध वापराच्या उदाहरणांची काही छोटी उदाहरणे यांची ओळख करून देईल.


## अनुक्रमणिका  

[आढावा](../../../../07-building-chat-applications/python)  
[OpenAI सेवा कशी वापरावी](../../../../07-building-chat-applications/python)  
[1. तुमची OpenAI सेवा तयार करणे](../../../../07-building-chat-applications/python)  
[2. स्थापना](../../../../07-building-chat-applications/python)    
[3. ओळखपत्रे](../../../../07-building-chat-applications/python)  

[वापराचे प्रकार](../../../../07-building-chat-applications/python)    
[1. मजकूराचा सारांश काढा](../../../../07-building-chat-applications/python)  
[2. मजकूर वर्गीकृत करा](../../../../07-building-chat-applications/python)  
[3. नवीन उत्पादनांची नावे तयार करा](../../../../07-building-chat-applications/python)  
[4. वर्गीकरणासाठी फाइन ट्यून करा](../../../../07-building-chat-applications/python)  

[संदर्भ](../../../../07-building-chat-applications/python)


### तुमचा पहिला प्रॉम्प्ट तयार करा  
ही छोटी सराव क्रिया तुम्हाला OpenAI मॉडेलला "सारांश" या सोप्या कामासाठी प्रॉम्प्ट कसे द्यायचे याची मूलभूत ओळख करून देईल.

**पायऱ्या**:  
1. तुमच्या python वातावरणात OpenAI लायब्ररी इंस्टॉल करा  
2. नेहमीच्या सहाय्यक लायब्ररी लोड करा आणि तुम्ही तयार केलेल्या OpenAI सेवेसाठी सुरक्षा credentials सेट करा  
3. तुमच्या कामासाठी एक मॉडेल निवडा  
4. मॉडेलसाठी एक सोपा प्रॉम्प्ट तयार करा  
5. तुमची विनंती मॉडेल API ला सबमिट करा!


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )


### 3. योग्य मॉडेल शोधणे  
GPT-3.5-turbo किंवा GPT-4 मॉडेल्स नैसर्गिक भाषा समजू शकतात आणि तयार करू शकतात.


In [ ]:
# Select the General Purpose curie model for text
model = "gpt-3.5-turbo"

## 4. प्रॉम्प्ट डिझाइन

"मोठ्या भाषा मॉडेल्समधील जादू म्हणजे, प्रचंड प्रमाणात मजकूरावर ही मॉडेल्स भविष्यवाणीतील त्रुटी कमी करण्यासाठी प्रशिक्षण घेतात, त्यामुळे या मॉडेल्सना या भविष्यवाण्यांसाठी उपयुक्त संकल्पना शिकायला मिळतात. उदाहरणार्थ, त्यांना अशा संकल्पना शिकायला मिळतात जशा की"(1):

* शब्दांची योग्य स्पेलिंग
* व्याकरण कसे कार्य करते
* वाक्य कसे बदलायचे (पॅराफ्रेज)
* प्रश्नांची उत्तरे कशी द्यायची
* संभाषण कसे चालवायचे
* अनेक भाषांमध्ये लेखन कसे करायचे
* कोड कसा लिहायचा
* इत्यादी

#### मोठ्या भाषा मॉडेलला कसे नियंत्रित करावे  
"मोठ्या भाषा मॉडेलसाठी सर्वात प्रभावी इनपुट म्हणजे मजकूर प्रॉम्प्ट आहे(1).

मोठ्या भाषा मॉडेल्सना काही पद्धतींनी आउटपुट देण्यासाठी प्रॉम्प्ट करता येते:

सूचना: मॉडेलला तुम्हाला काय हवे आहे ते सांगा
पूर्णता: तुम्हाला हवे असलेले सुरुवातीचे वाक्य पूर्ण करण्यासाठी मॉडेलला प्रवृत्त करा
डेमोन्स्ट्रेशन: मॉडेलला तुम्हाला काय हवे आहे ते दाखवा, यासाठी:
प्रॉम्प्टमध्ये काही उदाहरणे द्या
फाइन-ट्यूनिंग ट्रेनिंग डेटासेटमध्ये शेकडो किंवा हजारो उदाहरणे द्या"

#### प्रॉम्प्ट तयार करताना तीन मूलभूत मार्गदर्शक तत्त्वे:

**दाखवा आणि सांगा**. तुम्हाला काय हवे आहे ते स्पष्ट करा, सूचना, उदाहरणे किंवा दोन्हींच्या संयोजनाने. जर तुम्हाला मॉडेलने वस्तूंची यादी वर्णानुक्रमाने लावावी किंवा परिच्छेदाचा भावनानुसार वर्गीकरण करावे असे वाटत असेल, तर ते तसेच दाखवा.

**उत्तम दर्जाचा डेटा द्या**. जर तुम्ही वर्गीकरण तयार करत असाल किंवा मॉडेलने काही पॅटर्न फॉलो करावा अशी अपेक्षा करत असाल, तर पुरेशी उदाहरणे द्या. तुमची उदाहरणे नीट तपासा — मॉडेल साधारणपणे इतके हुशार असते की साध्या स्पेलिंगच्या चुका ओळखू शकते आणि उत्तर देऊ शकते, पण कधी कधी ते मुद्दाम केले आहे असे गृहीत धरू शकते आणि त्यामुळे उत्तरावर परिणाम होऊ शकतो.

**तुमच्या सेटिंग्ज तपासा.** temperature आणि top_p या सेटिंग्ज ठरवतात की मॉडेल किती निश्चितपणे उत्तर देईल. जर तुम्ही असे उत्तर विचारत असाल ज्याचे एकच बरोबर उत्तर आहे, तर हे सेटिंग्ज कमी ठेवा. जर तुम्हाला विविध प्रकारची उत्तरे हवी असतील, तर ही सेटिंग्ज जास्त ठेवा. या सेटिंग्जबद्दल लोक सर्वात जास्त चूक करतात ती म्हणजे, हे "हुशारी" किंवा "सर्जनशीलता"चे नियंत्रण आहे असे समजतात.

स्रोत: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## मजकूराचा सारांश  
#### आव्हान  
मजकूराचा सारांश देताना मजकूराच्या शेवटी 'tl;dr:' जोडा. लक्षात घ्या की मॉडेल कोणतीही अतिरिक्त सूचना न देता अनेक कामे कशी करू शकते. तुम्ही tl;dr पेक्षा अधिक वर्णनात्मक सूचनांसह प्रयोग करून मॉडेलचे वर्तन बदलू शकता आणि तुम्हाला मिळणारा सारांश सानुकूलित करू शकता(3).

अलीकडील संशोधनात असे दिसून आले आहे की मोठ्या प्रमाणावर मजकूरावर पूर्व-प्रशिक्षण करून आणि नंतर विशिष्ट कामासाठी फाइन-ट्यूनिंग केल्याने अनेक NLP कामांमध्ये आणि बेंचमार्कमध्ये मोठी प्रगती झाली आहे. जरी ही पद्धत साधारणपणे काम-निरपेक्ष असली तरी, तरीही हजारो किंवा दहा हजारो उदाहरणांच्या काम-विशिष्ट फाइन-ट्यूनिंग डेटासेट्सची गरज असते. याउलट, माणसे साधारणपणे काही उदाहरणे किंवा सोप्या सूचनांमधून नवीन भाषा काम सहज करू शकतात - हे सध्याच्या NLP प्रणालींना अजूनही जमत नाही. येथे आम्ही दाखवतो की भाषा मॉडेल्सचे प्रमाण वाढवल्याने काम-निरपेक्ष, कमी उदाहरणांमधून काम करण्याची क्षमता मोठ्या प्रमाणात सुधारते, कधी कधी पूर्वीच्या अत्याधुनिक फाइन-ट्यूनिंग पद्धतींशी स्पर्धा देखील करू शकते.



Tl;dr


# विविध वापरांसाठी सराव  
1. मजकूराचा सारांश काढा  
2. मजकूर वर्गीकृत करा  
3. नवीन उत्पादनांची नावे तयार करा


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## मजकूर वर्गीकरण  
#### आव्हान  
वर्गीकरणाच्या वेळी दिलेल्या श्रेणींमध्ये घटक वर्गीकृत करा. खालील उदाहरणात, आम्ही वर्गीकरणासाठी श्रेणी आणि मजकूर दोन्ही प्रॉम्प्टमध्ये दिले आहेत (*playground_reference).

ग्राहक चौकशी: नमस्कार, माझ्या लॅपटॉपच्या कीबोर्डमधील एक की नुकतीच तुटली आहे आणि मला त्याचा बदल हवा आहे:

वर्गीकृत श्रेणी:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## नवीन उत्पादनांची नावे तयार करा
#### आव्हान
उदाहरणातील शब्दांपासून उत्पादनांची नावे तयार करा. येथे आम्ही ज्या उत्पादनासाठी नावे तयार करायची आहेत त्याची माहिती दिली आहे. आम्ही एक तत्सम उदाहरणही दिले आहे, ज्यातून आम्हाला कोणत्या प्रकारचा नमुना हवा आहे हे स्पष्ट होते. आम्ही temperature मूल्य जास्त ठेवले आहे जेणेकरून उत्तरांमध्ये जास्त कल्पकता आणि नाविन्य येईल.

उत्पादनाचे वर्णन: घरगुती मिल्कशेक बनवणारे यंत्र
बीज शब्द: जलद, आरोग्यदायी, कॉम्पॅक्ट.
उत्पादनांची नावे: होमशेकर, फिट शेकर, क्विकशेक, शेक मेकर

उत्पादनाचे वर्णन: असे बूट ज्यांचा आकार कोणत्याही पायाला बसू शकतो.
बीज शब्द: अनुकूल, फिट, ऑम्नी-फिट.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# संदर्भ  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Examples](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [GPT-3 ला मजकूर वर्गीकरणासाठी फाइन-ट्यून करण्याच्या सर्वोत्तम पद्धती](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# अधिक मदतीसाठी  
[OpenAI कमर्शियलायझेशन टीम](AzureOpenAITeam@microsoft.com)


# योगदानकर्ते
* Louis Li



---

**अस्वीकरण**:  
हे दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित केले आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील मूळ दस्तऐवज हा अधिकृत स्रोत मानावा. अत्यावश्यक माहितीसाठी, व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर केल्यामुळे उद्भवणाऱ्या कोणत्याही गैरसमजुती किंवा चुकीच्या अर्थ लावण्यास आम्ही जबाबदार राहणार नाही.
